In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool
load_dotenv()

True

In [2]:
SERVICENOW_GET_RECORDS_PROMPT = """
    This tool is a wrapper around the ServiceNow table API, useful when you need to search for records in ServiceNow.
    Records can be of type Incident, Change Requests or Service Requests.
    The input to the tool is a query string that filters results based on attribute value.
    For example, to find all the Incident records with Priority value of 1, you would 
    pass the following string: priority=1
    """
SERVICENOW_INC_CREATE_PROMPT = """
    This tool is a wrapper around the ServiceNow incident table API, useful when you need to create a ServiceNow incident. 
    The input to this tool is a dictionary specifying the fields of the ServiceNow incident, and will be passed to the table API call.
    For example, to create an incident called "test issue" with description "test description", you would pass in the following dictionary: 
    {{"short_description": "test issue", "description": "test description"}}
    """

In [3]:
os.environ['SERVICENOW_FILTER'] = "active=true"
os.environ['SERVICENOW_FIELDS'] = "number,short_description,description"

In [4]:
from snow_api_wrapper import ServiceNowAPIWrapper
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper

servicenow = ServiceNowAPIWrapper()
duckduck_search = DuckDuckGoSearchAPIWrapper()

In [5]:
tools = [
    Tool.from_function(
        name = "ServiceNow_Get",
        func=servicenow.get_incidents,
        description=SERVICENOW_GET_RECORDS_PROMPT
    ),
    Tool.from_function(
        name = "ServiceNow_Create",
        func=servicenow.create_incident,
        description=SERVICENOW_INC_CREATE_PROMPT
    ),
    Tool(
        name = "Search_DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

In [6]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/opt/anaconda3/lib/python3.12/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
response = agent_executor.invoke({"input":"You are a technical support analyst. Retrieve a list of ServiceNow incidents of priority 1 and for each incident, use its description to find a solution on the web."})

In [ ]:
print (response)

In [ ]:
response = agent_executor.invoke({"input":"Show me the solutions for other incidents"})

In [ ]:
response = agent_executor.invoke({"input":"Search the web for solutions to the above incidents"})


In [7]:
response = agent_executor.invoke({"input":"You are a technical support analyst and a customer has just called about a faulty laptop. Create a ServiceNow incident for this issue."})



> Entering new AgentExecutor chain...

Invoking: `ServiceNow_Create` with `{"short_description": "Faulty laptop issue", "description": "Customer reported an issue with a faulty laptop. Please investigate and resolve.", "caller_id": "Customer"}`


<Response [201]>I have successfully created a ServiceNow incident for the faulty laptop issue reported by the customer.

> Finished chain.
